In [ ]:
import numpy as np
import pandas as pd
import math
from numba import jit
import matplotlib.pyplot as plt
from operator import itemgetter

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Kaggle PRML/Dataset/train.csv')
songs = pd.read_csv('/content/drive/MyDrive/Kaggle PRML/Dataset/songs.csv')
sfl = pd.read_csv('/content/drive/MyDrive/Kaggle PRML/Dataset/save_for_later.csv')
song_labels = pd.read_csv('/content/drive/MyDrive/Kaggle PRML/Dataset/song_labels.csv')

In [ ]:
def song_info(songid, songs):
  rows = songs.loc[songs["song_id"] == songid]
  return rows.iloc[0]

In [ ]:
def tuple_distance(training_inst, test_inst):
  year_dist = 0
  try:
    year_dist = (float(training_inst["released_year"]) - float(test_inst["released_year"]))**2
  except:
    pass
  #language_dist = 0 if training_inst["language"] == test_inst["language"] else 1
  comments_dist = (float(training_inst["number_of_comments"]) - float(test_inst["number_of_comments"]))**2
  distance = math.sqrt(year_dist + comments_dist)
  return (training_inst, distance)

def get_neighbours(training_set, test_inst, k):
  distances = [tuple_distance(training_inst, test_inst) for index,training_inst in training_set.iterrows()]
  sorted_distances = sorted(distances, key=itemgetter(1))
  sorted_training_instances = [tuple[0] for tuple in sorted_distances]
  return sorted_training_instances[:k]

In [ ]:
def get_neighbour_cust(c_id, sfl_check):
  sfl1 = np.tile(sfl_check[c_id][:], (14053, 1))
  result = np.multiply(sfl_check,sfl1)
  res = np.sum(result, axis=1)
  neighbours = np.argsort(res)
  return neighbours[14047:]

In [ ]:
cust_avg = train.groupby("customer_id")["score"].mean()
song_avg = train.groupby("song_id")["score"].mean()

In [ ]:
customer_dict = {}
for i,cust in enumerate(train["customer_id"].unique()):
  customer_dict[cust] = i

In [ ]:
scores = np.zeros((14053, 10001))
sfl_check = np.zeros((14053, 10001))
for index,row in train.iterrows():
  s_id = int(row["song_id"])
  c_id = int(customer_dict[row["customer_id"]])
  scores[c_id][s_id] = row["score"]

for index,row in sfl.iterrows():
  s_id = int(row["song_id"])
  c_id = int(customer_dict[row["customer_id"]])
  sfl_check[c_id][s_id] = 1

In [ ]:
norm_songs = songs.copy()
norm_songs["released_year"] = norm_songs["released_year"]/norm_songs["released_year"].abs().max()
norm_songs["number_of_comments"] = norm_songs["number_of_comments"]/norm_songs["number_of_comments"].abs().max()

In [ ]:
test_inst = song_info(972, norm_songs)
for i in get_neighbours(norm_songs, test_inst, 5):
  print(i["song_id"])

1526
2355
1477
706
1088


In [ ]:
ans = 0
dis = 0
for n in neighbours:
  s_id = n[0]["song_id"]
  #print(s_id)
  c_id = int(customer_dict["I24143"])
  #print(c_id)
  score1 = scores[c_id][s_id]
  if score1 != 0:
    ans += score1/n[1]
    dis += (1/n[1])
print(ans/dis)

3.5831395208352146


In [ ]:
get_neighbour_cust(int(customer_dict["M38457"]), sfl_check)

array([13053, 12632, 12666,  1578, 10420, 13979])

In [ ]:
temp = songs["song_id"].value_counts()
print((temp))
print(len(temp))

In [ ]:
temp = np.zeros((3, 3))
temp1 = np.zeros((3, 3))
temp[1][0] = 1
temp[1][2] = 1
temp[2][1] =1
temp1[1][0] = 1
temp1[1][2] = 1
temp1[2][1] =1
print(temp)
print(np.sum(temp, axis=1))
res = np.sum(temp, axis=1)
print(np.argsort(res))

[[0. 0. 0.]
 [1. 0. 1.]
 [0. 1. 0.]]
[0. 2. 1.]
[0 2 1]
